In [2]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from skimage import transform

In [3]:
#load model
model_emotion = model_from_json(open("fer.json", "r").read())
model_gender = model_from_json(open("gender.json", "r").read())
#load weights
model_emotion.load_weights("fer_wts.h5")
model_gender.load_weights("gender_wts.h5")

In [4]:
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read() # captures frame and returns boolean value and captured image
    if not ret:
        continue
    color_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.03, 10, minSize=(180,180))

    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x-10,y-10),(x+w+10,y+h+10),(255,0,0), thickness=1)
        roi_g = gray_img[y-20:y+h+20,x-20:x+w+20] # cropping region of interest i.e. face area from  image
        np_image_g = np.array(roi_g).astype('float32')/255
        np_image_g = transform.resize(np_image_g, (48, 48, 1))
        np_image_g = np.expand_dims(np_image_g, axis=0)

        predicted_emotion = model_emotion.predict(np_image_g)
        
        roi_c = color_img[y-20:y+h+20,x-20:x+w+20] # cropping region of interest i.e. face area from  image
        np_image_c = np.array(roi_c).astype('float32')/255
        np_image_c = transform.resize(np_image_c, (48, 48, 3))
        np_image_c = np.expand_dims(np_image_c, axis=0)
        
        predicted_gender = model_gender.predict(np_image_c)[0][0]

        #find max indexed array
        max_index = np.argmax(predicted_emotion[0])
        
        emotions = ('Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral')
        
        if predicted_gender > 0.5:
            gender = "F"
        else:
            gender = "M"
        
        predicted = emotions[max_index] + ', ' + gender

        cv2.putText(test_img, predicted, (int(x-10), int(y-10)), cv2.FONT_HERSHEY_SIMPLEX, .8, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (800, 550))
    cv2.imshow('Facial emotion analysis ',resized_img)

    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>